In [ ]:
# creating the classes
import keyboard as kb

class HauntedMansion:
    def __init__(self):
        self.rooms = {}
        self.doors = {}

    def add_room(self, room):
        self.rooms[room.name] = room

    def add_door(self, door):
        if door.room1.name in self.rooms and door.room2.name in self.rooms:
            self.doors[door.name] = door
            door.room1.add_door(door)
            door.room2.add_door(door)

class Room:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.objects = {}
        self.doors = []

    def add_door(self, door):
         if door not in self.doors:
             self.doors.append(door)

    def add_object(self, obj):
        self.objects[obj.name] = obj

    def search(self):
        if self.objects:
            print("You found:")
            for obj in self.objects.values():
                print(f"- {obj.name}: {obj.description}")
        else:
            print("There's nothing interesting here.")
        if self.doors:
             print(f"You also find that there are {len(self.doors)} doors:")
             for door in self.doors:
                print(f"- {door.name}")
        else:
             print("There are no doors here.")

class Object:
    def __init__(self, name, description, item=None):
        self.name = name
        self.description = description
        self.item = item

    def interact(self, player):
        if self.item:
            print(f"You found a {self.item.name} inside the {self.name}!")
            player.inventory.append(self.item)
            self.item = None  # The item is taken
        else:
            print(f"The {self.name} is empty.")

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

class Door:
    def __init__(self, name, room1, room2, key_required = None):
        self.name = name
        self.room1 = room1
        self.room2 = room2
        self.key_required = key_required

    def get_other_room(self, current_room):
        return self.room2 if current_room == self.room1 else self.room1

class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []
        self.current_action = None

    def game_loop(self):
        print(f"\nYou are in {self.current_room.name}. {self.current_room.description}")
        instructions = "Use 'W' to move, 'Q' to search, 'E' to interact, 'Esc' to quit."
        further_instructions = "To move through a door or interact with an object, please press the corresponding number of your choice."
        print(instructions + " " + further_instructions)
        print("Use 'H' to display instructions again at any point.")
        while True:
            event = kb.read_event()
            if event.event_type == "down":
                if event.name == "w":
                    self.move()
                elif event.name == "q":
                    self.search_room()
                elif event.name == "e":
                    self.interact_with_object()
                elif event.name == "h":
                    if self.current_action == "moving" or self.current_action == "interacting":
                        print(further_instructions)
                    else:
                        print(instructions)
                elif event.name == "esc":
                    print("Exiting game...")
                    break

    def move(self):
        if self.current_action:
            return
        
        self.current_action = "moving"
        doors = self.current_room.doors
        if not doors:
            print("There are no doors here. How did you even get here?")
            self.current_action = None
            return
        
        print("\nWhere do you want to move?")
        for index, door in enumerate(doors, start=1):
            print(f"{index} - {door.name}")

        while True:
            for index, door in enumerate(doors, start=1):
                if kb.is_pressed(str(index)):
                    selected_door = doors[index - 1]
                    other_room = selected_door.get_other_room(self.current_room)
                    if selected_door.key_required and selected_door.key_required not in self.inventory:
                        print(f"You need a {selected_door.key_required.name} to go there.")
                    else:
                        self.current_room = other_room
                        print(f"You moved to {other_room.name}. {other_room.description}")

                    self.current_action = None
                    return
    
    def search_room(self):
        self.current_room.search()

    def interact_with_object(self):
        if self.current_action:
            return
        
        self.current_action = "interacting"
        objects = self.current_room.objects
        if not objects:
            print("There's nothing to interact with here.")
            self.current_action = None
            return

        print("\nWhat do you want to interact with?")
        for index, obj in enumerate(objects.values(), start=1):
            print(f"{index} - {obj.name}")
        while True:
            for index in range(1, len(objects) + 1):
                if kb.is_pressed(str(index)):
                    obj = list(objects.values())[index - 1]
                    obj.interact(self)
                    self.current_action = None
                    return

In [ ]:
# creating & setting the game

# Create a house
house = HauntedMansion()

# Create rooms
kitchen = Room("Kitchen", "A warm kitchen with a smell of fresh bread.")
hallway = Room("Hallway", "An empty hallway.")
bedroom = Room("Bedroom", "A peaceful bedroom with a soft bed.")
dining_room = Room("Dining Room", "A nice room to have a meal.")
outside = Room("Outside", "A breeze of fresh air and singing birds. Congratulations, you've made it to the outside and won the game!")

silver_key = Item("Silver key", "Opens the door between the dining room and the kitchen")
golden_key = Item("Golden key", "Opens the door between the hallway and the dining room")
master_key = Item("Master key", "Opens the big door")


# Add rooms to the house
house.add_room(kitchen)
house.add_room(hallway)
house.add_room(bedroom)
house.add_room(dining_room)
house.add_room(outside)


silver_door = Door("Silver door", dining_room, kitchen, silver_key)
white_door = Door("White door", dining_room, hallway)
gold_door = Door("Gold door", hallway, bedroom, golden_key)
big_door = Door("Big door", hallway, outside, master_key)



# Connect rooms with doors
house.add_door(silver_door)
house.add_door(white_door)
house.add_door(gold_door)
house.add_door(big_door)


# house.add_door("Kitchen", "Dining Room", key_required="silver key")
# house.add_door("Dining Room", "Hallway")  # No key required for this door
# house.add_door("Hallway", "Bedroom", key_required="gold key")
# house.add_door("Hallway", "Outside", key_required="master key")

# Create a player and set their starting room
player = Player()
player.current_room = dining_room

# Create a drawer that contains a key
chest = Object("Chest", "A mysterious chest.", golden_key)
drawer = Object("Drawer", "An old wooden drawer.", silver_key)
bed = Object("Bed", "A large bed.", master_key)

# Add the drawer to the kitchen
dining_room.add_object(drawer)
kitchen.add_object(chest)
bedroom.add_object(bed)

In [ ]:
player.game_loop()